In [1]:
import sys
sys.path.append("/projects/pw8/wl45/open3spn2")
sys.path.append("/projects/pw8/wl45/openawsem")
sys.path.append("/Users/weilu/open3spn2")
sys.path.append("/Users/weilu/openmmawsem")
sys.path.append("C:/Users/luwei/Documents/GitHub/open3spn2")
sys.path.append("C:/Users/luwei/Documents/GitHub/openawsem")


import open3SPN2
import ffAWSEM
import time
import simtk.openmm
import simtk.openmm.app
import numpy as np

In [6]:
import os
from Bio.Seq import reverse_complement
os.environ["X3DNA"] = "/Users/weilu/Research/build/x3dna-v2.4"
# /projects/pw8/wl45/build/x3dna-v2.4

In [72]:
from Bio.PDB.Polypeptide import three_to_one

def convert_to_one(resName):
    DNA_resNames = ["DA", "DC", "DT", "DG"]
    if resName in DNA_resNames:
        return resName[1]
    else:
        return three_to_one(resName)
def isDNARes(resName):
    DNA_resNames = ["DA", "DC", "DT", "DG"]
    return resName in DNA_resNames

def writePDB(atoms,pdb_file):
    with open(pdb_file, 'w+') as pdb:
        for i, atom in atoms.iterrows():
            pdb_line = f'{atom.recname:<6}{atom.serial:>5} {atom["name"]:^4}{atom.altLoc:1}'+\
                    f'{atom.resname:<3} {atom.chainID:1}{atom.resSeq:>4}{atom.iCode:1}   '+\
                    f'{atom.x:>8.3f}{atom.y:>8.3f}{atom.z:>8.3f}' +\
                    f'{atom.occupancy:>6.2f}{atom.occupancy:>6.2f}'+' ' * 10 +\
                    f'{atom.element:>2}{atom.charge:>2}'
            assert len(pdb_line) == 80, f'An item in the atom table is longer than expected ({len(pdb_line)})\n{pdb_line}'
            pdb.write(pdb_line + '\n')

In [ ]:
original_pdb_file = "/Users/weilu/Research/server/jun_week3_2020/automatic_man_made_DNA/original_pdbs/6cta.pdb"
with open(original_pdb_file) as f:
    a = f.readlines()
    
seq_dic = {}
seq_length_dic = {}
is_DNA_chain = {}
for line in a:
    if line[:6] != "SEQRES":
        continue
        # print(line)
    ch = line[11]
    line_index = int(line[7:10])
    n = int(line[13:17])
    if line_index == 1:
        seq_dic[ch] = ""
        seq_length_dic[ch] = n
    seq = "".join([convert_to_one(one) for one in line[19:].split()])
    is_DNA_chain[ch] = np.alltrue([isDNARes(one) for one in line[19:].split()])
    seq_dic[ch] += seq
    # print(f"-{ch}-{line_index}-")
    # print(line, seq)
DNA_chain = []
for ch, length  in seq_length_dic.items():
    # print(length, ch)
    assert length == len(seq_dic[ch])
    if is_DNA_chain[ch]:
        DNA_chain.append(seq_dic[ch])
        

first_chain_seq = DNA_chain[0]
second_chain_seq = DNA_chain[1]
assert reverse_complement(first_chain_seq[1:]) == second_chain_seq[1:]
# Initialize the DNA from a sequence.
# DNA type can be changed to 'A' or 'B'
new_seq = "A"*100 + first_chain_seq + "A"*100
seq = new_seq
dna=open3SPN2.DNA.fromSequence(seq,dna_type='B_curved')

# Compute the topology for the DNA structure.
# Since the dna was generated from the sequence using X3DNA,
# it is not necesary to recompute the geometry.

dna.computeTopology(template_from_X3DNA=False)
writePDB(dna.atoms, "/Users/weilu/Research/server/jun_week3_2020/automatic_man_made_DNA/setups/DNA_part.pdb")
# dna.writePDB("/Users/weilu/Research/server/jun_week3_2020/automatic_man_made_DNA/setups/DNA_part.pdb")

In [98]:
DNA_resNames = ["DA", "DC", "DT", "DG"]
pdb_file = "/Users/weilu/Research/server/jun_week3_2020/automatic_man_made_DNA/6cta_with_long_DNA.pdb"
dna=open3SPN2.DNA.fromCoarsePDB(pdb_file)
new_dna = dna.atoms.reset_index()
new_dna_dna_only = new_dna.query("resname in @DNA_resNames")
pdb_file = "/Users/weilu/Research/server/jun_week3_2020/automatic_man_made_DNA/setups/with_formated_DNA_6cta_clean.pdb"
# seq_file = "/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/1mnn_with_long_DNA.seq"
dna=open3SPN2.DNA.fromCoarsePDB(pdb_file)


start_of_second_chain =  int((new_dna_dna_only["resSeq"].max())/2)
bond_length_cutoff=1.8
data = dna.atoms
bonds_list = []

interacting_atom = ["A", "C", "G", "T"]
a = data.query("resname in @DNA_resNames").query("name in @interacting_atom").reset_index(drop=True)
n_atom = len(a)
for i in range(n_atom):
    dna_res_1 = a.iloc[i]
    chainID_1 = dna_res_1["chainID"]
    for j in range(i+1, n_atom):
        dna_res_2 = a.iloc[j]
        chainID_2 = dna_res_2["chainID"]
        if chainID_1 == chainID_2:
            continue

        atom1_pos = dna_res_1[["x", "y", "z"]].values
        atom2_pos = dna_res_2[["x", "y", "z"]].values
        dis = atom1_pos - atom2_pos
        r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
        r /= 10   # convert to nm
        if r > bond_length_cutoff:
            # bong length larger than r will be ignored.
            continue
        # print(i, j, r)
        name = dna_res_1["name"]
        resSeq = dna_res_1["resSeq"] + 100
        target_atom = new_dna_dna_only.query(f"resSeq=={resSeq} and name=='{name}'")
        if len(target_atom) != 1:
            print(target_atom)
        else:
            serial_1 = int(target_atom["index"])

        name = dna_res_2["name"]
        resSeq = dna_res_2["resSeq"] + start_of_second_chain + 100
        target_atom = new_dna_dna_only.query(f"resSeq=={resSeq} and name=='{name}'")
        if len(target_atom) != 1:
            # print(target_atom)
            pass
        else:
            serial_2 = int(target_atom["index"])
#         bonds_list.append([dna_res_1["serial"], dna_res_2["serial"], r])
        bonds_list.append([serial_1, serial_2, r])
    
bond_info = pd.DataFrame(bonds_list, columns=["atom1", "atom2", "r"])
# bond_info.to_csv("/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/bond_info_1mnn_with_long_DNA.csv")


In [116]:
start_of_second_chain

217

In [113]:
import pandas as pd


In [114]:
bond_info

,atom1,atom2,r
0,308,976,1.575230
1,308,976,1.298570
2,308,976,1.013989
3,308,976,0.858007
4,308,976,0.682224
...,...,...,...
119,344,958,0.722997
120,344,958,0.892758
121,344,958,1.120073
122,344,967,1.440017


In [87]:
writePDB(dna.atoms, "/Users/weilu/Research/server/jun_week3_2020/automatic_man_made_DNA/setups/DNA_part.pdb")

In [85]:
dna.atoms

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,type
0,ATOM,0,A,,DA,A,1,,-0.440158,2.425299,0.012213,0,0,N,,A
1,ATOM,1,S,,DA,A,1,,-2.789750,6.611468,-0.768648,0,0,H,,S
2,ATOM,2,A,,DA,A,2,,-1.740121,1.565361,3.230875,0,0,N,,A
3,ATOM,3,P,,DA,A,2,,-5.581382,7.212560,1.561305,0,0,P,,P
4,ATOM,4,S,,DA,A,2,,-6.079404,3.595565,2.391771,0,0,H,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,ATOM,1295,P,,DT,B,433,,5.099871,-7.850242,5.365716,0,0,P,,P
1296,ATOM,1296,S,,DT,B,433,,1.512616,-7.149802,4.489658,0,0,H,,S
1297,ATOM,1297,T,,DT,B,434,,-0.035655,-3.380030,0.019816,0,0,S,,T
1298,ATOM,1298,P,,DT,B,434,,-0.345058,-9.264648,1.839537,0,0,P,,P


In [ ]:
cmd = "python ~/openmmawsem/mm_create_project.py ../../../../jun_week1_2020/protein_DNA_benchmark/cleaned_pdbs/1mnn.pdb"
# cmd = "cp ../only_protein/setups/1mnn/1mnn.pdb ."
cmd = "cp ../only_protein/setups/1mnn/1mnn-openmmawsem.pdb ."
cmd = "gg.py -d jun21 -m 1"
cmd = "pdb_chain -C shifted_1mnn.pdb > shifted_1mnn_become_chain_C.pdb"
cmd = "grep 'ATOM' shifted_1mnn_become_chain_C.pdb > Protein_part.pdb"
cmd = "rm 1mnn_with_long_DNA.pdb"
cmd = "cat DNA_part.pdb >> 1mnn_with_long_DNA.pdb"
cmd = "cat Protein_part.pdb >> 1mnn_with_long_DNA.pdb"
# get seq from 1mnn.pdb
cmd = "mv single_frags_1mnn.mem single_frags_1mnn_with_long_DNA.mem"
# do("gg_server.py -d jun21 -m 1")
# cmd = "python setup.py 1mnn_with_long_DNA --source ."

In [84]:
len(first_chain_seq)

17

In [42]:
# read the DNA seqeunce from the crystal structure.
pdb_file = "/Users/weilu/Research/server/jun_week3_2020/protein_DNA/DNAProtein_Platform_OpenCL_date_20200226_pdb_1mnn_repetition_0_clean.pdb"
dna=open3SPN2.DNA.fromCoarsePDB(pdb_file)

In [51]:
a = dna.getSequences().reset_index()

In [54]:
chain_list = a["level_0"].unique()

In [56]:
assert len(chain_list) == 2

In [57]:
chain_list[0]

'A'

In [58]:
chain_list[1]

'B'

In [72]:
first_chain_seq = "".join(a.query(f"level_0=='{chain_list[0]}'")[0].to_list())
second_chain_seq = "".join(a.query(f"level_0=='{chain_list[1]}'")[0].to_list())

In [73]:
first_chain_seq

'TGCGACACAAAAAC'

In [81]:
assert reverse_complement(first_chain_seq[1:]) == second_chain_seq[1:]

In [84]:
new_seq

'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAATGCGACACAAAAACAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'

In [87]:
# Initialize the DNA from a sequence.
# DNA type can be changed to 'A' or 'B'
new_seq = "A"*100 + first_chain_seq + "A"*100
seq = new_seq
dna=open3SPN2.DNA.fromSequence(seq,dna_type='B_curved')

# Compute the topology for the DNA structure.
# Since the dna was generated from the sequence using X3DNA,
# it is not necesary to recompute the geometry.

dna.computeTopology(template_from_X3DNA=False)


In [88]:
dna.writePDB("/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/DNA_part.pdb")

'/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/DNA_part.pdb'

In [ ]:
cleaned_pdb_filename = f"{name}-cleaned.pdb"
input_pdb_filename = f"{name}-openmmawsem.pdb"
do(f"grep 'ATOM' {input_pdb_filename} > tmp.pdb")
do(f"grep 'HETATM' {cleaned_pdb_filename} >> tmp.pdb")
do(f"mv tmp.pdb {input_pdb_filename}")


In [ ]:
cmd = "python ~/openmmawsem/mm_create_project.py ../../../../jun_week1_2020/protein_DNA_benchmark/cleaned_pdbs/1mnn.pdb"
# cmd = "cp ../only_protein/setups/1mnn/1mnn.pdb ."
cmd = "cp ../only_protein/setups/1mnn/1mnn-openmmawsem.pdb ."
cmd = "gg.py -d jun21 -m 1"
cmd = "pdb_chain -C shifted_1mnn.pdb > shifted_1mnn_become_chain_C.pdb"
cmd = "grep 'ATOM' shifted_1mnn_become_chain_C.pdb > Protein_part.pdb"
cmd = "rm 1mnn_with_long_DNA.pdb "
cmd = "cat DNA_part.pdb >> 1mnn_with_long_DNA.pdb"
cmd = "cat Protein_part.pdb >> 1mnn_with_long_DNA.pdb"
# get seq from 1mnn.pdb
cmd = "mv single_frags_1mnn.mem single_frags_1mnn_with_long_DNA.mem"
# do("gg_server.py -d jun21 -m 1")
# cmd = "python setup.py 1mnn_with_long_DNA --source ."

In [83]:
pdb_file = "/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/1mnn_with_long_DNA.pdb"
seq_file = "/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/1mnn_with_long_DNA.seq"
dna=open3SPN2.DNA.fromCoarsePDB(pdb_file)
new_dna = dna.atoms.reset_index()

In [65]:
with open(seq_file) as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB(pdb_file, sequence=protein_sequence_one)

In [80]:
protein_resNames = ["NGP", "IGL", "IPR", "NTER", "CTER"]
DNA_resNames = ["DA", "DC", "DT", "DG"]
group1_index = []
group2_index = []
data = protein.atoms.reset_index()
for i, line in data.iterrows():
    resname = line["resname"]
    index = line["index"]
    name_ = line["name"]
    if resname in DNA_resNames and name_ == "P":
        group1_index.append(index)
    if resname in protein_resNames and name_ == "CA":
        group2_index.append(index)

In [78]:
data = dna.atoms.reset_index()

In [85]:
new_dna_dna_only = new_dna.query("resname in @DNA_resNames")

In [86]:
new_dna_dna_only

,index,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,type
0,0,ATOM,1,A,,DA,A,1,,-0.440,2.425,0.012,0.0,0.0,N,,A
1,1,ATOM,2,S,,DA,A,1,,-2.790,6.611,-0.769,0.0,0.0,H,,S
2,2,ATOM,3,A,,DA,A,2,,-1.740,1.565,3.231,0.0,0.0,N,,A
3,3,ATOM,4,P,,DA,A,2,,-5.581,7.213,1.561,0.0,0.0,P,,P
4,4,ATOM,5,S,,DA,A,2,,-6.079,3.596,2.392,0.0,0.0,H,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277,1277,ATOM,1278,P,,DT,B,427,,5.100,-7.850,5.366,0.0,0.0,P,,P
1278,1278,ATOM,1279,S,,DT,B,427,,1.513,-7.150,4.490,0.0,0.0,H,,S
1279,1279,ATOM,1280,T,,DT,B,428,,-0.036,-3.380,0.020,0.0,0.0,S,,T
1280,1280,ATOM,1281,P,,DT,B,428,,-0.345,-9.265,1.840,0.0,0.0,P,,P


In [89]:
pdb_file = "/Users/weilu/Research/server/jun_week3_2020/protein_DNA/DNAProtein_Platform_OpenCL_date_20200226_pdb_1mnn_repetition_0_clean.pdb"
# seq_file = "/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/1mnn_with_long_DNA.seq"
dna=open3SPN2.DNA.fromCoarsePDB(pdb_file)

In [90]:
k = 1
bond_length_cutoff=1.8
data = dna.atoms
bonds_list = []
DNA_resNames = ["DA", "DC", "DT", "DG"]
interacting_atom = ["A", "C", "G", "T"]
a = data.query("resname in @DNA_resNames").query("name in @interacting_atom").reset_index(drop=True)
n_atom = len(a)
for i in range(n_atom):
    dna_res_1 = a.iloc[i]
    chainID_1 = dna_res_1["chainID"]
    for j in range(i+1, n_atom):
        dna_res_2 = a.iloc[j]
        chainID_2 = dna_res_2["chainID"]
        if chainID_1 == chainID_2:
            continue
        assert chainID_1 == "A"
        assert chainID_2 == "B"
        atom1_pos = dna_res_1[["x", "y", "z"]].values
        atom2_pos = dna_res_2[["x", "y", "z"]].values
        dis = atom1_pos - atom2_pos
        r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
        r /= 10   # convert to nm
        if r > bond_length_cutoff:
            # bong length larger than r will be ignored.
            continue
        # print(i, j, r)
        name = dna_res_1["name"]
        resSeq = dna_res_1["resSeq"] + 100
        target_atom = new_dna_dna_only.query(f"resSeq=={resSeq} and name=='{name}'")
        if len(target_atom) != 1:
            print(target_atom)
        else:
            serial_1 = int(target_atom["index"])

        name = dna_res_2["name"]
        resSeq = dna_res_2["resSeq"] + 313
        target_atom = new_dna_dna_only.query(f"resSeq=={resSeq} and name=='{name}'")
        if len(target_atom) != 1:
            print(target_atom)
        else:
            serial_2 = int(target_atom["index"])
#         bonds_list.append([dna_res_1["serial"], dna_res_2["serial"], r])
        bonds_list.append([serial_1, serial_2, r])

Empty DataFrame
Columns: [index, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [index, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [index, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [index, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [index, recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []


In [91]:
import pandas as pd
bond_info = pd.DataFrame(bonds_list, columns=["atom1", "atom2", "r"])
bond_info.to_csv("/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/bond_info_1mnn_with_long_DNA.csv")

In [ ]:
a = pd.read_csv("/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/bond_info_1mnn_with_long_DNA.csv", index_col=0)

for i, line in a.iterrows():
    atom1 = line["atom1"]
    atom2 = line["atom2"]
    r = line["r"]
    print(atom1, atom2, r)

In [40]:
new_dna.query("resSeq==101 and name=='T'")

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,type
299,ATOM,300,T,,DT,A,101,,-14.457,8.865,326.23,0.0,0.0,S,,T


In [41]:
if chainID == "A":
    resSeq = 1 + 100
    int(new_dna.query("resSeq==101 and name=='T'")["serial"])
if chainID == "B":
    resSeq = 1 + 313
    int(new_dna.query("resSeq==101 and name=='T'")["serial"])

300

In [52]:
new_dna = new_dna.query("resname in @DNA_resNames")

In [88]:
new_dna_dna_only

,index,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,type
0,0,ATOM,1,A,,DA,A,1,,-0.440,2.425,0.012,0.0,0.0,N,,A
1,1,ATOM,2,S,,DA,A,1,,-2.790,6.611,-0.769,0.0,0.0,H,,S
2,2,ATOM,3,A,,DA,A,2,,-1.740,1.565,3.231,0.0,0.0,N,,A
3,3,ATOM,4,P,,DA,A,2,,-5.581,7.213,1.561,0.0,0.0,P,,P
4,4,ATOM,5,S,,DA,A,2,,-6.079,3.596,2.392,0.0,0.0,H,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1277,1277,ATOM,1278,P,,DT,B,427,,5.100,-7.850,5.366,0.0,0.0,P,,P
1278,1278,ATOM,1279,S,,DT,B,427,,1.513,-7.150,4.490,0.0,0.0,H,,S
1279,1279,ATOM,1280,T,,DT,B,428,,-0.036,-3.380,0.020,0.0,0.0,S,,T
1280,1280,ATOM,1281,P,,DT,B,428,,-0.345,-9.265,1.840,0.0,0.0,P,,P


In [53]:
k = 1
bond_length_cutoff=1.8
data = dna.atoms
bonds_list = []
DNA_resNames = ["DA", "DC", "DT", "DG"]
interacting_atom = ["A", "C", "G", "T"]
a = data.query("resname in @DNA_resNames").query("name in @interacting_atom").reset_index(drop=True)
n_atom = len(a)
for i in range(n_atom):
    dna_res_1 = a.iloc[i]
    chainID_1 = dna_res_1["chainID"]
    for j in range(i+1, n_atom):
        dna_res_2 = a.iloc[j]
        chainID_2 = dna_res_2["chainID"]
        if chainID_1 == chainID_2:
            continue
        assert chainID_1 == "A"
        assert chainID_2 == "B"
        atom1_pos = dna_res_1[["x", "y", "z"]].values
        atom2_pos = dna_res_2[["x", "y", "z"]].values
        dis = atom1_pos - atom2_pos
        r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
        r /= 10   # convert to nm
        if r > bond_length_cutoff:
            # bong length larger than r will be ignored.
            continue
        # print(i, j, r)
        name = dna_res_1["name"]
        resSeq = dna_res_1["resSeq"] + 100
        target_atom = new_dna_dna_only.query(f"resSeq=={resSeq} and name=='{name}'")
        if len(target_atom) != 1:
            print(target_atom)
        else:
            serial_1 = int(target_atom["index"])

        name = dna_res_2["name"]
        resSeq = dna_res_2["resSeq"] + 313
        target_atom = new_dna_dna_only.query(f"resSeq=={resSeq} and name=='{name}'")
        if len(target_atom) != 1:
            print(target_atom)
        else:
            serial_2 = int(target_atom["index"])
#         bonds_list.append([dna_res_1["serial"], dna_res_2["serial"], r])
        bonds_list.append([serial_1, serial_2, r])

Empty DataFrame
Columns: [recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []
Empty DataFrame
Columns: [recname, serial, name, altLoc, resname, chainID, resSeq, iCode, x, y, z, occupancy, tempFactor, element, charge, type]
Index: []


In [61]:
a = pd.read_csv("/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/bond_info_1mnn_with_long_DNA.csv", index_col=0)

In [63]:
for i, line in a.iterrows():
    atom1 = line["atom1"]
    atom2 = line["atom2"]
    r = line["r"]
    print(atom1, atom2, r)

300.0 965.0 1.7669660240083849
300.0 968.0 1.4419823403911711
300.0 971.0 1.2091323500758715
300.0 974.0 0.8749505757469962
300.0 977.0 0.7367425805530723
303.0 962.0 1.6036632876012349
303.0 965.0 1.353613319969924
303.0 968.0 1.0386278736872032
303.0 971.0 0.8429822358745174
303.0 974.0 0.5601379651478732
303.0 977.0 0.6097265452643506
306.0 959.0 1.619531815062612
306.0 962.0 1.2704311905805838
306.0 965.0 1.0613017525661583
306.0 968.0 0.7928380225493729
306.0 971.0 0.6846149720828487
306.0 974.0 0.5271372971057915
306.0 977.0 0.7406163716796976
309.0 956.0 1.6124163017037505
309.0 959.0 1.4113481391917444
309.0 962.0 1.0945667544741164
309.0 965.0 0.8713648260057323
309.0 968.0 0.6870952917900114
309.0 971.0 0.5725176765830033
309.0 974.0 0.5682101547843015
309.0 977.0 0.8411220957744483
312.0 953.0 1.6348273395071418
312.0 956.0 1.3210408358563337
312.0 959.0 1.146639555396551
312.0 962.0 0.8668838215124334
312.0 965.0 0.6488138562022241
312.0 968.0 0.5919374460194258
312.0 971.0

In [49]:
dna_res_1["name"]

'C'

In [ ]:
def dna_constraint_term(dna, k=10*kilocalorie_per_mole, bond_length_cutoff=0.65, forceGroup=19):
    print("dna_constraint_term is on")
    k = k.value_in_unit(kilojoule_per_mole)
    data = dna.atoms
    bonds_list = []
    DNA_resNames = ["DA", "DC", "DT", "DG"]
    interacting_atom = ["A", "C", "G", "T"]
    a = data.query("resname in @DNA_resNames").query("name in @interacting_atom").reset_index(drop=True)
    n_atom = len(a)
    for i in range(n_atom):
        dna_res_1 = a.iloc[i]
        chainID_1 = dna_res_1["chainID"]
        for j in range(i+1, n_atom):
            dna_res_2 = a.iloc[j]
            chainID_2 = dna_res_2["chainID"]
            if chainID_1 == chainID_2:
                continue
            atom1_pos = dna_res_1[["x", "y", "z"]].values
            atom2_pos = dna_res_2[["x", "y", "z"]].values
            dis = atom1_pos - atom2_pos
            r = (dis[0]**2 + dis[1]**2 + dis[2]**2)**0.5
            r /= 10   # convert to nm
            if r > bond_length_cutoff:
                # bong length larger than r will be ignored.
                continue
            # print(i, j, r)
            bonds_list.append([dna_res_1["serial"], dna_res_2["serial"], r])
    bondForce = CustomBondForce(f"{k}*(r-r0)^2")
    bondForce.addPerBondParameter('r0')
    for (atom1, atom2, r) in bonds_list:
        # print(atom1, atom2, r)
        bondForce.addBond(int(atom1), int(atom2), [r])

In [ ]:
cmd = "cp ../only_protein/setups/1mnn/1mnn-openmmawsem.pdb ."
cmd = "pdb_chain -C 1mnn-openmmawsem.pdb > 1mnn_chain_C.pdb"
cmd = "cat DNA_part.pdb >> 1mnn_with_long_DNA.pdb"
cmd = "cat 1mnn_chain_C.pdb >> 1mnn_with_long_DNA.pdb"


In [7]:
from pathlib import Path
home = str(Path.home())

In [8]:
home

'/Users/weilu'

In [70]:
first_chain_seq

['T', 'G', 'C', 'G', 'A', 'C', 'A', 'C', 'A', 'A', 'A', 'A', 'A', 'C']

In [85]:
second_chain_seq

'AGTTTTTGTGTCGC'

In [68]:
from Bio.Seq import reverse_complement

In [69]:
reverse_complement("ATCG")

'CGAT'

In [66]:
Bio.Seq.reverse_complemente("ATCG")

AttributeError: module 'Bio.Seq' has no attribute 'reverse_complemente'

In [8]:
# read the DNA seqeunce from the crystal structure.
pdb_file = "/Users/weilu/Research/server/jun_week3_2020/protein_DNA/setups/p_20200619_pdb_4y60_repetition_0_clean.pdb"
dna=open3SPN2.DNA.fromCoarsePDB(pdb_file)

In [34]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import pandas as pd

In [43]:
pdb = PDBFile(pdb_file)

In [44]:
a = pdb.getTopology()

In [5]:
# Initialize the DNA from a sequence.
# DNA type can be changed to 'A' or 'B'

seq='ATACAAAGGTGCGAGGTTTCTATGCTCCCACG'
dna=open3SPN2.DNA.fromSequence(seq,dna_type='B_curved')

# Compute the topology for the DNA structure.
# Since the dna was generated from the sequence using X3DNA,
# it is not necesary to recompute the geometry.

dna.computeTopology(template_from_X3DNA=False)

# Create the system.
# To set periodic boundary conditions (periodicBox=[50,50,50]).
# The periodic box size is in nanometers.
dna.periodic=False
s=open3SPN2.System(dna, periodicBox=None)

#Add 3SPN2 forces
s.add3SPN2forces(verbose=True)

Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics


In [7]:
dna.writePDB("/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/test.pdb")

'/Users/weilu/Research/server/jun_week3_2020/man_made_DNA/test.pdb'

In [10]:
import scipy.spatial.distance as sdist
import pandas as pd


In [ ]:
fix=open3SPN2.fixPDB(pdb)

In [3]:
pdb_file = "/Users/weilu/Research/server/jun_week1_2020/protein_DNA_benchmark/DNAProtein_Platform_OpenCL_date_20200226_pdb_1a36_repetition_0_clean.pdb"
seq_file = "/Users/weilu/Research/server/jun_week1_2020/protein_DNA_benchmark/DNAProtein_Platform_OpenCL_date_20200226_pdb_1a36_repetition_0_protein.seq"
with open(seq_file) as ps:
    protein_sequence_one=ps.readlines()[0]
protein=ffAWSEM.Protein.fromCoarsePDB(pdb_file,sequence=protein_sequence_one)

In [6]:
data = protein.atoms

In [27]:
protein_resNames = ["NGP", "IGL", "IPR", "NTER", "CTER"]
DNA_resNames = ["DA", "DC", "DT", "DG"]
group1_index = []
group2_index = []
for i, line in data.iterrows():
    resname = line["resname"]
    index = line["serial"]
    name = line["name"]
    if resname in DNA_resNames and name == "P":
        group1_index.append(index)
    if resname in protein_resNames and name == "CA":
        group2_index.append(index)

In [11]:

#Calculate native distances
CA_atoms=protein.atoms[protein.atoms.name=='CA']
P_atoms=protein.atoms[protein.atoms.name=='P']
d=sdist.cdist(CA_atoms[['x','y','z']],P_atoms[['x','y','z']])/10 #Distance in nanometers
d_sq=pd.DataFrame(d,index=CA_atoms.index,columns=P_atoms.index)
d=d_sq.copy()
d['Protein']=d.index
d=d.melt(id_vars=['Protein'])
d=d.rename(columns={'variable': 'DNA', 'value': 'distance'})

In [23]:
d.query("distance < 1")

,Protein,DNA,distance
352,2209,3,0.993812
353,2215,3,0.982950
354,2221,3,0.909546
356,2233,3,0.853868
357,2238,3,0.817565
...,...,...,...
20757,2315,116,0.692107
20758,2321,116,0.960002
20759,2327,116,0.996391
21132,1272,119,0.958682


In [17]:
d["DNA"].unique()

array([3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51,
       54, 57, 60, 63, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98, 101,
       104, 107, 110, 113, 116, 119, 122, 125, 128], dtype=object)

In [18]:
d.shape

(23142, 3)

In [19]:
23142/40

578.55

In [31]:
data = pd.read_csv("/Users/weilu/Research/server/jun_week3_2020/protein_DNA/selected.csv", index_col=0)

sampled = data.sample(6, random_state=28)
pdb_list = sampled.idcode.to_list()

In [41]:
pdb_list = sampled.idcode.to_list()

In [42]:
pdb_list = [a.lower() for a in pdb_list]

In [43]:
print(pdb_list)

['4y60', '5ke8', '1a1j', '5lxu', '1skn', '6a2h']


In [35]:

data.sample(6, random_state=0)

,Number of missing residues,Resolution,idcode,release_date,First,Chain DNA1 ID,Chain DNA1 Compound,Chain DNA2 ID,Chain DNA2 Compound,Chain Protein ID,Chain Protein Compound
45,11,3.10,6JHE,2020-01-01,3.10,b,dna (5'-d(p*tp*tp*gp*ap*ap*ap*cp*cp*tp*tp*t)-3'),c,dna (5'-d(*ap*ap*ap*gp*gp*tp*tp*tp*cp*ap*a)-3'),a,ecf rna polymerase sigma factor sigw
59,0,1.90,6QEC,2020-02-05,1.90,u,dna (5'-d(*ap*tp*tp*cp*gp*ap*ap*tp*ap*t*tp*ap*...,b,dna (5'-d(*ap*tp*tp*cp*gp*ap*ap*tp*ap*t*tp*ap*...,a,transcription factor lux
7,0,2.70,4S2Q,2016-02-17,2.70,a,dna (5'-d(p*ap*gp*gp*cp*tp*tp*tp*gp*tp*tp*cp*t...,b,dna (5'-d(p*ap*gp*gp*ap*gp*ap*ap*cp*ap*ap*ap*g...,d,transcription factor sox-9
50,7,2.75,5JLX,2016-06-22,2.75,"b, e",dna (5'-d(*ap*gp*ap*ap*ap*gp*cp*(c7s)p*ap*tp*t...,"c, f",dna (5'-d(*tp*cp*tp*cp*tp*ap*ap*tp*gp*gp*cp*tp...,"a, d",homeotic protein antennapedia
92,3,2.40,3U2B,2011-12-28,2.40,a,dna (5'-d(*gp*tp*cp*tp*cp*tp*ap*tp*tp*gp*tp*cp...,b,dna (5'-d(*cp*cp*ap*gp*gp*ap*cp*ap*ap*tp*ap*gp...,c,transcription factor sox-4
27,6,1.45,5KL3,2016-09-14,1.45,b,dna (5'-d(*ap*gp*cp*gp*tp*gp*gp*gp*ap*gp*t)-3'),c,dna (5'-d(*tp*ap*cp*tp*cp*cp*cp*ap*cp*gp*c)-3'),a,wilms tumor protein


In [95]:
from Bio.PDB.PDBParser import PDBParser


parser = PDBParser(QUIET=True)